# Table of contents
### Aggregated mean for order_number grouped by derparment_id
### Setting loyalty flag
### Setting flags for type of spenders
### Setting flag for spending regularity of a user
### Cleaning outliers and exporting

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import datasets
path = r'C:\Users\raque\Documents\08-2023 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_prods_merged_v2.pkl'))

In [3]:
ords_prods_merge.shape

(32404859, 21)

In [4]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,...,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices,merge2,price_range_loc,busiest_day,busiest_hour
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders
1,1,1,2398795,1,2,3,7,15.0,196,1,...,both,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Average orders
2,2,2,473747,1,3,3,12,21.0,196,1,...,both,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Most orders
3,3,3,2254736,1,4,4,7,29.0,196,1,...,both,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Average orders
4,4,4,431534,1,5,4,15,28.0,196,1,...,both,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Most orders


### Aggregated mean for order_number grouped by derparment_id

In [5]:
# Group data by department_id and calculate the average orders from the column order_number
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### We can see now all the department_id's whether in the subset we could only see a sample. And the means we already had have changed slightly with all the data.

### Setting loyalty flag

In [6]:
# Creating new column with max orders a customer has made
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
# Checking it's been created 
ords_prods_merge.head(5)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,...,Unnamed: 0_y,product_name,aisle_id,department_id,prices,merge2,price_range_loc,busiest_day,busiest_hour,max_order
0,0,0,2539329,1,1,2,8,NaN,196,1,...,195,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10
1,1,1,2398795,1,2,3,7,15.0,196,1,...,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Average orders,10
2,2,2,473747,1,3,3,12,21.0,196,1,...,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Most orders,10
3,3,3,2254736,1,4,4,7,29.0,196,1,...,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Average orders,10
4,4,4,431534,1,5,4,15,28.0,196,1,...,195,Soda,77,7,9.0,both,Mid-range product,Slowest Days,Most orders,10


In [8]:
# If a customer has more than 40 orders is a Loyal Customer
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
# Customer with orders between 10 and 40 are Regular Customers
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
# Customers with less than 10 orders are New Customers
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Counting customers grouped by Loyalty Flag
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [12]:
# checking everything was interpreted correctly in the affected columns
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


#### Statistics of the product prices by loyalty flags

In [13]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['min', 'max', 'mean', 'median']})

prices                           
                    min      max       mean median
loyalty_flag                                      
Loyal customer      1.0  99999.0  10.386336    7.4
New customer        1.0  99999.0  13.294670    7.4
Regular customer    1.0  99999.0  12.495717    7.4

#### Regular and New customers have a very similar mean, which is a little bit higher than the Loyal customers. Loyal customers, may do more smaller shops, or profit more often from discounts or reward programs. 

### Setting flags for type of spenders

In [14]:
# Creating new column with average spent per user
ords_prods_merge['avg_spends'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
ords_prods_merge.head(15)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,...,aisle_id,department_id,prices,merge2,price_range_loc,busiest_day,busiest_hour,max_order,loyalty_flag,avg_spends
0,0,0,2539329,1,1,2,8,NaN,196,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797
1,1,1,2398795,1,2,3,7,15.0,196,1,...,77,7,9.0,both,Mid-range product,Slowest Days,Average orders,10,New customer,6.367797
2,2,2,473747,1,3,3,12,21.0,196,1,...,77,7,9.0,both,Mid-range product,Slowest Days,Most orders,10,New customer,6.367797
3,3,3,2254736,1,4,4,7,29.0,196,1,...,77,7,9.0,both,Mid-range product,Slowest Days,Average orders,10,New customer,6.367797
4,4,4,431534,1,5,4,15,28.0,196,1,...,77,7,9.0,both,Mid-range product,Slowest Days,Most orders,10,New customer,6.367797
5,5,5,3367565,1,6,2,7,19.0,196,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797
6,6,6,550135,1,7,1,9,20.0,196,1,...,77,7,9.0,both,Mid-range product,Busiest day,Most orders,10,New customer,6.367797
7,7,7,3108588,1,8,1,14,14.0,196,2,...,77,7,9.0,both,Mid-range product,Busiest day,Most orders,10,New customer,6.367797
8,8,8,2295261,1,9,1,16,0.0,196,4,...,77,7,9.0,both,Mid-range product,Busiest day,Most orders,10,New customer,6.367797
9,9,9,2550362,1,10,4,8,30.0,196,1,...,77,7,9.0,both,Mid-range product,Slowest Days,Average orders,10,New customer,6.367797


In [16]:
# If the mean of the prices of products purchased by a user is lower than 10, then is flagged as a “Low spender”
ords_prods_merge.loc[ords_prods_merge['avg_spends'] < 10, 'type_spender'] = 'Low spender'

In [17]:
# If the mean of the prices of products purchased by a user is higher than 10, then is flagged as a “Low spender”
ords_prods_merge.loc[ords_prods_merge['avg_spends'] >= 10, 'type_spender'] = 'High spender'

In [18]:
# Counting customers grouped by type of spender 
ords_prods_merge['type_spender'].value_counts(dropna = False)

type_spender
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [19]:
# Counting customers grouped by type of spender 
ords_prods_merge.groupby('type_spender').nunique()['user_id']

type_spender
High spender      5357
Low spender     200852
Name: user_id, dtype: int64

In [20]:
# checking everything was interpreted correctly in the affected columns
ords_prods_merge[['user_id', 'avg_spends', 'type_spender']].head(15)

,user_id,avg_spends,type_spender
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
5,1,6.367797,Low spender
6,1,6.367797,Low spender
7,1,6.367797,Low spender
8,1,6.367797,Low spender
9,1,6.367797,Low spender


### Setting flag for spending regularity of a user

In [21]:
# Creating new column with median of the days prior to an order
ords_prods_merge['median_days_prior'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [22]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,...,prices,merge2,price_range_loc,busiest_day,busiest_hour,max_order,loyalty_flag,avg_spends,type_spender,median_days_prior
0,0,0,2539329,1,1,2,8,NaN,196,1,...,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,1,1,2398795,1,2,3,7,15.0,196,1,...,9.0,both,Mid-range product,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.5
2,2,2,473747,1,3,3,12,21.0,196,1,...,9.0,both,Mid-range product,Slowest Days,Most orders,10,New customer,6.367797,Low spender,20.5
3,3,3,2254736,1,4,4,7,29.0,196,1,...,9.0,both,Mid-range product,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.5
4,4,4,431534,1,5,4,15,28.0,196,1,...,9.0,both,Mid-range product,Slowest Days,Most orders,10,New customer,6.367797,Low spender,20.5


In [23]:
# If the median of “days_since_last_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_prior'] > 20, 'order_regularity'] = 'Non-frequent customer'

In [24]:
# If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
ords_prods_merge.loc[(ords_prods_merge['median_days_prior'] > 10) & (ords_prods_merge['median_days_prior'] <= 20) , 'order_regularity'] = 'Regular customer'

In [25]:
# If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_prior'] <= 10, 'order_regularity'] = 'Frequent customer'

In [26]:
# Counting customers grouped by type of spender 
ords_prods_merge['order_regularity'].value_counts(dropna = False)

order_regularity
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [27]:
# Counting customers grouped by type of spender 
ords_prods_merge.groupby('order_regularity').nunique()['user_id']

order_regularity
Frequent customer        86596
Non-frequent customer    59619
Regular customer         59993
Name: user_id, dtype: int64

In [28]:
# checking everything was interpreted correctly in the affected columns
ords_prods_merge[['user_id', 'median_days_prior', 'order_regularity']].head(15)

,user_id,median_days_prior,order_regularity
0,1,20.5,Non-frequent customer
1,1,20.5,Non-frequent customer
2,1,20.5,Non-frequent customer
3,1,20.5,Non-frequent customer
4,1,20.5,Non-frequent customer
5,1,20.5,Non-frequent customer
6,1,20.5,Non-frequent customer
7,1,20.5,Non-frequent customer
8,1,20.5,Non-frequent customer
9,1,20.5,Non-frequent customer


### Cleaning outliers and exporting

In [29]:
# Cleaning outliers from prices
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [30]:
# Export data to pkl

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_prods_merged_v3.pkl'))